In [1]:
%%capture
!pip install datasets bitsandbytes trl==0.16.1

In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset

template_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

def format_prompt(example):
  chat = example['messages']
  prompt = template_tokenizer.apply_chat_template(chat,tokenize=False)

  return {'text':prompt}

dataset = (
    load_dataset("HuggingFaceH4/ultrachat_200k",  split="test_sft")
      .shuffle(seed=42)
      .select(range(3_000))
)

dataset = dataset.map(format_prompt).remove_columns(['messages'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [7]:
dataset['text'][2576]

'<|user|>\nGiven the text: Knock, knock. Who’s there? Hike.\nCan you continue the joke based on the given text material "Knock, knock. Who’s there? Hike"?</s>\n<|assistant|>\nSure! Knock, knock. Who\'s there? Hike. Hike who? Hike up your pants, it\'s cold outside!</s>\n<|user|>\nCan you tell me another knock-knock joke based on the same text material "Knock, knock. Who\'s there? Hike"?</s>\n<|assistant|>\nOf course! Knock, knock. Who\'s there? Hike. Hike who? Hike your way over here and let\'s go for a walk!</s>\n'

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

# 4-비트 양자화 설정 - QLoRA의 Q 단계
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-비트 정밀도 모델 로드
    bnb_4bit_quant_type="nf4",  # 양자화 종류
    bnb_4bit_compute_dtype="float16",  # 계산 dtype
    bnb_4bit_use_double_quant=True,  # 이중 양자화 적용
)

# 모델을 로드하고 GPU에서 훈련합니다.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",

    # 일반적인 SFT에서는 다음을 삭제하세요.
    quantization_config=bnb_config,
)
model.config.use_cache = False

# LLaMA 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha = 128,
    lora_dropout= 0.1,
    r=64,
    bias = 'none',
    task_type = 'CAUSAL_LM',
    target_modules =
    ['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [11]:
from trl import SFTConfig

output_dir = './results'

training_arguments = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True,
    dataset_text_field="text",
    max_length=512
)

In [12]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset = dataset,
    processing_class=tokenizer,
    args = training_arguments,
    peft_config=peft_config
)

trainer.train()
trainer.model.save_pretrained("TinyLlama-1.1B-qlora")

Converting train dataset to ChatML:   0%|          | 0/3000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jakeswj33 (jakeswj33-korea-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.579900
20,1.445600
30,1.428700
40,1.465800
50,1.454700
60,1.369800
70,1.480000
80,1.433400
90,1.415200
100,1.387500


In [14]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora",
    low_cpu_mem_usage=True,
    device_map="auto",
)

# LoRA와 베이스 모델을 병합합니다.
merged_model = model.merge_and_unload()

In [15]:
from transformers import pipeline

# 사전에 정의된 프롬프트 템플릿을 사용합니다.
prompt = """<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
"""

# 인스트럭션 튜닝된 모델을 실행합니다.
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])

Device set to use cuda:0


<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
Large Language Models (LLMs) are machine learning models that are trained to generate natural language. LLMs are becoming increasingly popular in the field of natural language processing (NLP) because they can be used to generate and analyze large amounts of text.

One of the main advantages of LLMs is their ability to generate natural-sounding text. This is achieved through the use of deep neural networks, which are capable of learning to generate sequences of words or phrases that sound and feel natural to humans.

Another advantage of LLMs is their ability to generate high-quality summaries of long texts. This is achieved through the use of contextualized representations, which are used to summarize long texts by looking at the context surrounding each word or phrase.

The development of LLMs has led to the creation of new applications in various fields, including marketing, finance, and medicine. LLMs are als

In [16]:
from datasets import load_dataset

def format_prompt(example):
    """TinyLlama의 <|user|> 템플릿을 사용해 프롬프트를 구성합니다"""

    # 템플릿 포맷팅
    system = "<|system|>\n" + example['system'] + "</s>\n"
    prompt = "<|user|>\n" + example['input'] + "</s>\n<|assistant|>\n"
    chosen = example['chosen'] + "</s>\n"
    rejected = example['rejected'] + "</s>\n"

    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# 데이터셋에 템플릿을 적용하고 비교적 짧은 대답을 선택합니다
dpo_dataset = load_dataset("argilla/distilabel-intel-orca-dpo-pairs", split="train")
dpo_dataset = dpo_dataset.filter(
    lambda r:
        r["status"] != "tie" and
        r["chosen_score"] >= 8 and
        not r["in_gsm8k_train"]
)
dpo_dataset = dpo_dataset.map(format_prompt, remove_columns=dpo_dataset.column_names)
dpo_dataset

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/79.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12859 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12859 [00:00<?, ? examples/s]

Map:   0%|          | 0/5922 [00:00<?, ? examples/s]

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 5922
})

In [17]:
from peft import AutoPeftModelForCausalLM
from transformers import BitsAndBytesConfig, AutoTokenizer

# 4-비트 양자화 설정 - QLoRA의 Q 단계
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-비트 정밀도 모델 로드
    bnb_4bit_quant_type="nf4",  # 양자화 종류
    bnb_4bit_compute_dtype="float16",  # 계산 dtype
    bnb_4bit_use_double_quant=True,  # 이중 양자화 적용
)

# LoRA와 베이스 모델을 합칩니다.
model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora",
    low_cpu_mem_usage=True,
    device_map="auto",
    quantization_config=bnb_config,
)
merged_model = model.merge_and_unload()

# LLaMA 토크나이저를 로드합니다.
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [18]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA 설정
peft_config = LoraConfig(
    lora_alpha=128,  # LoRA 스케일링
    lora_dropout=0.1,  # LoRA 층의 드롭아웃
    r=64,  # 랭크
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=  # 대상 층
     ['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

# 훈련을 위해 모델을 준비합니다.
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [19]:
from trl import DPOConfig

output_dir = "./results"

# 훈련 매개변수
training_arguments = DPOConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True,
    warmup_ratio=0.1,
    beta=0.1,
    max_prompt_length=512,
    max_length=512
)

In [ ]:
from trl import DPOTrainer

# DPOTrainer 객체를 만듭니다.
dpo_trainer = DPOTrainer(
    model,
    args=training_arguments,
    train_dataset=dpo_dataset,
    processing_class=tokenizer,
    peft_config=peft_config
)

# DPO로 모델을 미세 튜닝합니다.
dpo_trainer.train()

# 어댑터를 저장합니다.
dpo_trainer.model.save_pretrained("TinyLlama-1.1B-dpo-qlora")

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Extracting prompt in train dataset:   0%|          | 0/5922 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/5922 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5922 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.691100
20,0.653700
30,0.553300
40,0.594700
50,0.561600


In [1]:
from peft import PeftModel

# LoRA와 베이스 모델을 합칩니다.
model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora",
    low_cpu_mem_usage=True,
    device_map="auto",
)
sft_model = model.merge_and_unload()

# DPO LoRA와 SFT 모델을 합칩니다.
dpo_model = PeftModel.from_pretrained(
    sft_model,
    "TinyLlama-1.1B-dpo-qlora",
    device_map="auto",
)
dpo_model = dpo_model.merge_and_unload()

NameError: name 'AutoPeftModelForCausalLM' is not defined

In [ ]:
from transformers import pipeline

# 사전에 정의된 프롬프트 템플릿을 사용합니다.
prompt = """<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
"""

# 인스트럭션 튜닝된 모델을 실행합니다.
pipe = pipeline(task="text-generation", model=dpo_model, tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])